In [1]:
%load_ext autoreload
%autoreload 2

import dask
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize
import pymc as pm
import arviz as az
import corner
import pickle
import cloudpickle

from normalization_utils import fit_combo_scaler, inverse_combo_scaler

np.random.seed(42)

### Select dataset to work with here

Uncomment one of these lines only to define the dataset we're working with.

Key to BedMachine Masks:

0. Ocean
1. Ice-free land
2. Grounded
3. Floating

In [2]:
# # CReSIS Greenland Grounded
# dataset_name, ice_sheet, bm_mask_whitelist, input_data_path = (
#     "cresis_gis_grounded", "greenland", [2], 
#     "../data_preprocessing/snr_data_cresis_gis_with_inputs.nc"
# )

# # UTIG Antarctica Grounded
# dataset_name, ice_sheet, bm_mask_whitelist, input_data_path = (
#     "utig_ais_grounded", "antarctica", [2], 
#     "../data_preprocessing/snr_data_utig_ais_with_inputs.nc"
# )

# # CReSIS Antarctica Grounded
# dataset_name, ice_sheet, bm_mask_whitelist, input_data_path = (
#     "cresis_ais_grounded", "antarctica", [2], 
#     "../data_preprocessing/snr_data_cresis_ais_with_inputs.nc"
# )

# CReSIS Antarctica Floating
dataset_name, ice_sheet, bm_mask_whitelist, input_data_path = (
    "cresis_ais_floating", "antarctica", [3], 
    "../data_preprocessing/snr_data_cresis_ais_with_inputs.nc"
)

In [ ]:
# Remove any missing values and subset to variables of interest
input_df = xr.open_dataset(input_data_path).to_dataframe()

# Keep only the variables of interest
vars_of_interest = ['snr', 'thickness', 'speed', 't2m', 'surface', 'bm_mask']
if 'picked_thickness' in input_df.columns:
    vars_of_interest.append('picked_thickness')
input_df = input_df[vars_of_interest]
input_df = input_df.dropna()

# Add log(speed)
input_df['log_speed'] = np.log(input_df['speed'])
input_df = input_df[~np.isinf(input_df['log_speed'])]

# Filter by BedMachine mask
input_df = input_df[input_df['bm_mask'].isin(bm_mask_whitelist)]

# Filter out zero thickness picks if available
if 'picked_thickness' in input_df.columns:
    input_df = input_df[input_df['picked_thickness'] > 0]

# UTIG SNR is inverted
if 'utig' in dataset_name:
    input_df['snr'] = -input_df['snr']

input_df

In [ ]:
vars_to_norm = ['snr', 'thickness', 't2m', 'surface']

# Normalize inputs and outputs
normalization_parameters = {}
for var in vars_to_norm:
    input_df[var + "_norm"], normalization_parameters[var] = fit_combo_scaler(input_df[var])

# Plot histograms with/without normalization

fig, axs = plt.subplots(len(vars_to_norm), 2, figsize=(6, 1.5 * len(vars_to_norm)))
for i, var in enumerate(vars_to_norm):
    axs[i, 0].hist(input_df[var], bins=20)
    axs[i, 0].set_title(f"{var} histogram")
    axs[i, 1].hist(input_df[var + "_norm"], bins=20)
    axs[i, 1].set_title(f"{var}_norm histogram")

fig.tight_layout()

### Model training

In [5]:
def ln_like(theta, thickness_obs, surf_temp_obs, surf_elev_obs, snr_sim):
    beta_0, beta_thickness, beta_surf_temp, beta_surf_elev = theta
    model = beta_0 * np.exp(beta_thickness*thickness_obs +
                            beta_surf_temp*surf_temp_obs +
                            beta_surf_elev*surf_elev_obs
                           )
    
    ln_z = -0.5 * np.sum((snr_sim - model)**2 + np.log(2*np.pi))
    
    return ln_z

In [6]:
nll = lambda *args: -ln_like(*args)
initial = np.repeat(0.0,4) + 0.1 * np.random.randn(4)
likelihood = scipy.optimize.minimize(nll, initial, 
                      args=(input_df['thickness_norm'], input_df['t2m_norm'],
                            input_df['surface_norm'], input_df['snr_norm']), 
                      method='BFGS')
beta_0_ml, beta_thickness_ml, beta_surf_temp_ml, beta_surf_elev_ml = likelihood.x

In [ ]:
model = pm.Model()

with model:

    # Define weakly informative Normal priors for Ridge regression
    sigma = pm.HalfNormal("sigma", 1) #recommended for StudentT
    b0 = pm.Normal("intercept", beta_0_ml, sigma=10)
    b1 = pm.Normal("beta_thickness", beta_thickness_ml, sigma=10)
    b2 = pm.Normal("beta_surface_temp", beta_surf_temp_ml, sigma=10)
    b3 = pm.Normal("beta_surface_elev", beta_surf_elev_ml, sigma=10)
    # b_EX = pm.Normal("name", 0, sigma=0.1) #heavily regularizing prior, might be useful for colinear situations ahead
    
    d1 = pm.Data("thickness_norm", input_df['thickness_norm'])
    d2 = pm.Data("t2m_norm", input_df['t2m_norm'])
    d3 = pm.Data("surface_norm", input_df['surface_norm'])

    # Define linear model
    y_est = b0 + b1*d1 + b2*d2 + b3*d3

    # Define prior for StudentT degrees of freedom
    # Inverse Gamma is recommended
    nu = pm.InverseGamma("nu", alpha=1, beta=1)

    # Define Student T likelihood, because of the presence of outliers
    likelihood = pm.StudentT(
        "likelihood", mu=y_est, sigma=sigma, nu=nu, observed=input_df['snr_norm']
    )
    
    trace = pm.sample(1000, cores=3,
                          target_accept=0.8)

### Plot results

In [ ]:
# Corner plot
fig = corner.corner(np.vstack((trace.posterior['intercept'][0],
                               trace.posterior['beta_thickness'][0],
                               trace.posterior['beta_surface_temp'][0],
                               trace.posterior['beta_surface_elev'][0],
                               trace.posterior['sigma'][0])).T, 
                    labels = ['intercept', 'thickness', 
                              'T$_{surf}$', 'Elevation$_{surf}$',
                              '$\sigma$'], color='#1f77b4',
                    alpha=0.25, 
                    label_kwargs={"fontsize": 22, "labelpad": 5}, 
                    quantiles=[0.05, 0.5, 0.95])

In [ ]:
az.summary(trace, hdi_prob = 0.68, round_to=3).loc[['intercept', 
                                                    'beta_thickness', 
                                                    'beta_surface_temp',
                                                    'beta_surface_elev', 
                                                    'sigma'], :]

In [ ]:
az.plot_trace(trace, 
              var_names=['intercept', 'beta_thickness', 
                         'beta_surface_temp', 'beta_surface_elev', 
                         'sigma'], 
              combined=False, compact=False)
plt.tight_layout()

In [11]:
results = {
    'dataset_name': dataset_name,
    'ice_sheet': ice_sheet,
    'bm_mask_whitelist': bm_mask_whitelist,
    'input_data_path': input_data_path,
    'trace': trace,
    'model': model,
    'normalization_parameters': normalization_parameters
}

# Save the model
with open(f"outputs/{dataset_name}_model.pickle", "wb") as f:
    cloudpickle.dump(results, f)